In [ ]:
import sys, os
import copy
import shutil
import pathlib
import time
import importlib
import joblib
import pickle
import itertools
from tqdm import tqdm

import numpy as np
import cv2
import pandas as pd
import open3d as o3d

import scipy.stats as stats
from scipy.linalg import null_space
from scipy.sparse import csr_matrix
from scipy import interpolate

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
%load_ext autoreload
%autoreload 2

if not os.path.abspath("../") in sys.path:
    sys.path.append(os.path.abspath("../"))
    sys.path.append(os.path.abspath("../src"))

from src import normalization
from src import Camera, metashape_Camera
from src import camera_correspondence, FF_mat, epipole_angle, pair_and_key_gen, coll_dict_gen, pair_pt_gen
from src import FR_check, coordinate_dict_gen
from src import min_dist, tri, TDlines_gen, reprojection_gen, gen_support_dict, gen_support
from src import all_D, frag_list_fraged, all_sep

In [ ]:
def support_th(tags, temp_folder, sup_th):
    curve_fragment = []
    for tag in tags:
        if tag[1] == "R":
            continue

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.TDlines".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            lines_list = pickle.load(f)

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.support_dict".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            support_list, support_ac = pickle.load(f)
        #support_list, support_ac = support_dict[tag][0], support_dict[tag][1]
        tag_list = []
        for col, sup_col, ac_col in zip(lines_list, support_list, support_ac):
            col_curve_fragment = []
            for frag, sup, sup_ac in zip(col, sup_col, ac_col):
                if sup > sup_th:
                    frag = np.reshape(frag,(-1, 3))
                    frag = np.array([i for i,j in zip(frag, sup_ac >sup_th) if j])
                    col_curve_fragment.append(frag)
            tag_list.append(col_curve_fragment)
        curve_fragment.append(tag_list)
    return curve_fragment

def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry

angle = [0, np.pi/2, np.pi/2] # pcd.rotate()
angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])


def curve_fragment_align(curve_fragment):
    cfs = [[] for i in range(8)]
    for cols in curve_fragment:
        for i in range(len(cols)):
            cfs[i] += cols[i]
    temp_cfs = []
    for i, cf_label in enumerate(cfs):
        A = [frag  for frag in cf_label if frag.size!=0]
        temp_cfs.append(A)
    cfs = temp_cfs
    cfs = [list(map(lambda x: x*10@R@R_mirror, cf_label)) for cf_label in cfs]
    return cfs

def get_pts(mesh, edges, max_point_num=500):
    vertices = np.asarray(mesh.vertices)
    edges = np.asarray(edges)
    dist = 0
    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]
        end = vertices[edges][i][1]
        dist_part = np.sqrt(np.sum(np.square(start - end)))
        dist += dist_part
    unit_dist = dist/max_point_num
    next_start_length = 0
    next_start_partation = 0
    pts_list = []
    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]
        end = vertices[edges][i][1]
        next_st_pt = start + (end - start)*next_start_partation
        dist_part = np.sqrt(np.sum(np.square(start - end)))
        dist_nx = np.sqrt(np.sum(np.square(next_st_pt - end)))
        pts_num = int(dist_nx/unit_dist)+1
        pts = next_st_pt + np.repeat(np.arange(pts_num),3).reshape((pts_num,3)) * (end - next_st_pt)/(pts_num-1)
        end_pts = pts[-1]
        next_start_length = pts_num*unit_dist - dist_part + next_start_length
        next_start_partation = next_start_length/dist_part
        pts_list.append(pts)

    pts_array = np.concatenate(pts_list)

    return pts_array

def mesh_contours_load(mesh_path):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    edge_pts=[]
    for i in range(np.max(np.asarray(mesh.cluster_connected_triangles()[0]))+1):
        mesh = o3d.io.read_triangle_mesh(mesh_path)
        mesh.triangles = o3d.utility.Vector3iVector(np.asarray(mesh.triangles)[np.asarray(mesh.cluster_connected_triangles()[0])==i])
        #_, _, area = mesh.cluster_connected_triangles()
        edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)
        pts_array = get_pts(mesh, edges)
        edge_pts.append(pts_array)
    return edge_pts

def edge_pts_shuffle_core(cfs, edge_pts):
    cfs_ave = np.array([np.average(np.concatenate(cf_label), axis=0) for cf_label in cfs])
    edge_pts_ave = np.array([np.average(edge_pt, axis=0) for edge_pt in edge_pts])
    cfs_ave_bro = np.repeat(cfs_ave, len(edge_pts_ave), axis=0).reshape(len(cfs_ave), len(edge_pts_ave), 3)
    cfs2edge_pts_idx = np.argmin(np.sum(np.square(cfs_ave_bro - edge_pts_ave),axis=2), axis=0)
    temp_edge_pts = copy.deepcopy(edge_pts)
    for i, idx in enumerate(cfs2edge_pts_idx):
        temp_edge_pts[i] = edge_pts[idx]
    edge_pts = temp_edge_pts
    return edge_pts

def cf_evaluation_core(edge_pts_label, cfs_label, t_dist=0.05, t_part=0.8):
    rec_TF_list = []
    for i, cf in enumerate(cfs_label):
        edge_pt_bro = np.repeat(edge_pts_label, len(cf), axis=0).reshape(edge_pts_label.shape[0], len(cf), edge_pts_label.shape[1])
        all_dist = np.sqrt(np.sum(np.square(cf - edge_pt_bro), axis=2))
        min_dist_idx_rec = np.argmin(all_dist.T, axis=1)
        min_dist_idx_pre = np.argmin(all_dist, axis=1)
        if i == 0:
            temp_dist_pre = np.ones(min_dist_idx_pre.shape)*1000
        min_dist_rec = all_dist.T[np.arange(len(all_dist.T)), min_dist_idx_rec]
        min_dist_pre = all_dist[np.arange(len(all_dist)), min_dist_idx_pre]
        rec_TF = (np.sum(min_dist_rec<t_dist)/len(min_dist_rec))>t_part
        rec_TF_list.append(rec_TF)
        temp_dist_pre[np.where(temp_dist_pre>min_dist_pre)[0]] = min_dist_pre[np.where(temp_dist_pre>min_dist_pre)[0]]
    recall = sum(rec_TF_list)/len(rec_TF_list)
    precision = np.sum(temp_dist_pre<t_dist)/len(temp_dist_pre)
    return precision, recall

def cf_evaluation(edge_pts, cfs):
    leaves_precision = []
    leaves_recall = []
    for edge_pts_label, cfs_label in zip(edge_pts, cfs):
        precision, recall = np.nan, np.nan
        if len(cfs_label) != 0:
            precision, recall = cf_evaluation_core(edge_pts_label, cfs_label)
        leaves_precision.append(precision)
        leaves_recall.append(recall)
    return leaves_precision, leaves_recall

def edge_pts_shuffle(tags, mesh_path, img_num, temp_folder):
    curve_fragment = support_th(tags, temp_folder, 4)
    cfs = curve_fragment_align(curve_fragment)
    edge_pts = mesh_contours_load(mesh_path)
    edge_pts = edge_pts_shuffle_core(cfs, edge_pts)
    return edge_pts


def evaluation(tags, individual, img_num, temp_folder):
    precision_list = []
    recall_list = []
    edge_pts = edge_pts_shuffle(tags, mesh_path, img_num, temp_folder)
    if img_num == 128:
        sups = list(range(16,129))[::-4]
    if img_num == 64:
        sups = list(range(8,65))[::-2]
    if img_num == 32:
        sups = list(range(4,33))[::-1]

    for sup_th in sups:
        curve_fragment = support_th(tags, temp_folder, sup_th)
        cfs = curve_fragment_align(curve_fragment)
        precision, recall = cf_evaluation(edge_pts, cfs)
        precision_list.append(precision)
        recall_list.append(recall)
    return precision_list, recall_list

def get_tags(temp_folder):
    tags_path = list(temp_folder.glob("*.TDlines"))

    tags = []
    for tag_path in tags_path:
        tag_sp = tag_path.stem.split("_")
        tag_arr = ((int(tag_sp[0]), int(tag_sp[1])), tag_sp[2])
        tags.append(tag_arr)
    return tags

class Loader:
    def __init__(self, P_path, R_path, label_path, img_num):
        self.img_num = img_num
        
        with open(str(P_path), 'rb') as f:
            self.P = np.array(pickle.load(f))
        with open(str(R_path), 'rb') as f:
            self.R = np.array(pickle.load(f))
        with open(str(label_path), 'rb') as f:
            self.label = np.array(pickle.load(f))

    def prop_occ(self):
        label_mode, _ = stats.mode(self.label, axis=0)
        label_mode = np.squeeze(label_mode).astype("int")
        
    def prop_PR(self):
        sup_thes = list(range(4,33))[::-1]
        global_true_idx = [np.where((self.P>0.99)[:,i])[0]
                           if np.where((self.P>0.99)[:,i])[0].size != 0 
                           else np.array([-1])
                           for i in range(self.P.shape[1])]
        local_opt_idx = [np.argmax(self.R[:,i][global_true_idx[i]]) for i in range(len(global_true_idx))]
        global_opt_idx = [g_idx[l_idx] for g_idx, l_idx in zip(global_true_idx, local_opt_idx)]
        self.opt_sup_th = [sup_thes[opt_idx] for opt_idx in global_opt_idx]

In [ ]:
temp_folder = "TEMP"
img_num = 0 # IMAGE NUMBER
mesh_path = "MESH PATH"
output_file = "OPTIMISE CURVE FRAGMENTS FILE NAME"
label_path = "LABEL PATH"

temp_folder = pathlib.Path(temp_folder)

In [ ]:
# demo
temp_folder = "data/temp"
img_num = 32
mesh_path = "data/polygon/demo_poly.ply"
output_file = "demo_opt_cfs"
label_path = "data/labels/demo_poly_label.pickle"

temp_folder = pathlib.Path(temp_folder)

In [ ]:
tags = get_tags(temp_folder)
precision_list, recall_list = evaluation(tags, mesh_path, img_num, temp_folder)

os.makedirs('data/PR', exist_ok=True)

precision_path = os.path.join('data/PR',"{}_pre.pickle".format(img_num))
recall_path= os.path.join('data/PR',"{}_rec.pickle".format(img_num))

with open(precision_path, 'wb') as f:
    pickle.dump(precision_list, f)
with open(recall_path, 'wb') as f:
    pickle.dump(recall_list, f)

In [ ]:
output_file = "demo_opt_cfs"
label_path = "data/labels/demo_poly_label.pickle"

In [ ]:
lo = Loader(precision_path, recall_path, label_path, img_num)
lo.prop_occ()
lo.prop_PR()
opt_sup_list = lo.opt_sup_th

In [ ]:
tags = get_tags(temp_folder)

temp_cfs = []
for i, osth in enumerate(opt_sup_list):
    curve_fragment = support_th(tags, temp_folder, osth)
    cfs = curve_fragment_align(curve_fragment)
    temp_cfs.append(cfs[i])
    
os.makedirs("data/opt_curve_fragments", exist_ok=True)
with open(os.path.join("data/opt_curve_fragments", "{}.pickle".format(output_file)), 'wb') as f:
    pickle.dump(temp_cfs, f)